# As voltas de Liskov: iteradores em Python

<img src="Barbara-Liskov-Turing-Award-2008.webp"/>

Fonte: [MIT](http://www.pmg.csail.mit.edu/~liskov/)

## Homenageadas com o prêmio Turing até 2025

* **Frances E. Allen (2006):** teoria e prática de otimização de compiladores.
* **[Barbara Liskov (2008)](https://amturing.acm.org/award_winners/liskov_1108679.cfm):** inovações em linguagens de programação
* **Shafi Goldwasser (2012):** fundamentos teóricos da criptografia.

## A instrução `for`

### Problema

Escreva um programa que exibe os argumentos passados na linha de comando:

```shell
$ ./args açaí banana carambola
./args
açaí
banana
carambola
```

### C

```C
#include <stdio.h>

int main(int argc, char *argv[]) {
    for(int i = 0; i < argc; i++)
        printf("%s\n", argv[i]);
    return 0;
}
```

### Python

```python
import sys

for arg in sys.argv:
    print(arg)
```

### Citação

> Há 2 problemas difíceis em ciência da computação:<br>invalidar cache, nomear coisas e errar por 1.
<br>—*Leon Bambrick*

### Variantes de *erro-por-1* ao percorrer uma coleção em C:

1. laço termina no penúltimo item, o último é ignorado;
2. laço vai além do último item, acessando
memória inválida ([*segfault*](https://pt.wikipedia.org/wiki/Falha_de_segmenta%C3%A7%C3%A3o)).

A semântica do `for` em Python praticamente elimina esses erros.

## A solução geral: iteráveis e iteradores

### Iteráveis

**Comestível:** que pode ser comida

**Legível:** que pode ser lida

**Iterável:** que pode ser iterada (repetida)

Em programação, **iterável** é uma coleção que pode ser acessada item a item em uma estrutura de iteração.

Java não tinha o conceito de iterável até a versão 5 (2004). Python sempre teve, desde 1991.

### Iteráveis em Python

Iterável em Python é todo objeto que implementa:

* método `o.__iter__()` que devolve um iterador<br>e/ou...
* método `o.__getitem__(i)` que aceita índices a partir de `0`

Ex: `str` `bytes` `dict` `set` `array.array` `sqlite3.Cursor` `numpy.ndarray` ...

In [1]:
class Trinca:
    def __getitem__(self, i):
        return '🂱🃁🃑'[i]

In [2]:
for n in Trinca():
    print(n)

🂱
🃁
🃑


In [3]:
t = Trinca()
list(t)

['🂱', '🃁', '🃑']

### Iterador

Um objeto que implementa uma *interface padrão* para acessar o próximo item de uma coleção.

Em Python, essa interface tem um método: `__next__` (que invocamos através de `next(coleção)`).

### Gerador

```
iterador ≈ gerador  # sinônimos na comunidade Python
```

Uso _gerador_ quando se trata de séries infinitas. Ex: gerador de números de Fibonacci.

## O padrão de projeto Iterator

Fonte: [O Poder dos Geradores (16)](https://speakerdeck.com/ramalho/o-poder-dos-geradores?slide=16)

<img src="iterator-gof.png" style="width: 640px;"/>

<hr/>

Fonte: [O Poder dos Geradores (17)](https://speakerdeck.com/ramalho/o-poder-dos-geradores?slide=17)

<img src="iterator-poster.png" style="width: 640px;"/>

### Implementação "clássica" de Iterator em Python

(código não *pythônico* para ilustrar o padrão do GoF)

In [4]:
class Trem:
    def __init__(self, carros):
        self.carros = carros
    
    def __iter__(self):
        return IterTrem(self.carros)

class IterTrem:
    def __init__(self, carros):
        self.próximo = 0
        self.último = carros - 1
    
    def __next__(self):
        if self.próximo <= self.último:
            self.próximo += 1
            return f'carro #{(self.próximo)}'
        else:
            raise StopIteration()

In [5]:
trem = Trem(3)

for carro in trem:
    print(carro)

carro #1
carro #2
carro #3


### Citação

> Por exemplo, no mundo da orientação a objetos,<br>ouve-se muito sobre "padrões".<br>
> Me pergunto se esses padrões não são, às vezes,<br>evidências [...] do *compilador humano* em ação.<br>
> —*Paul Graham*

Fonte: [Are Design Patterns Missing Language Features?](https://wiki.c2.com/?AreDesignPatternsMissingLanguageFeatures)
(_Será que os padrões de projetos são recursos faltando em uma linguagem?_)

### Implementação pythônica

In [6]:
class Trem:
    def __init__(self, carros):
        self.carros = carros
    
    def __iter__(self):
        for n in range(self.carros):
            yield f'carro #{n}'

In [7]:
trem = Trem(3)

for carro in trem:
    print(carro)

carro #0
carro #1
carro #2


## Como funciona o `yield`

In [8]:
def gen123():
    ...

In [9]:
# for

In [10]:
# g = gen123()
# g

In [11]:
# list(g)

In [12]:
# list(g)

### Ping-pong: cliente x iterador

In [13]:
def genAB():
    ...

### Uma classe iterável

In [14]:
class Trinca:
    def __iter__(self):
        for carta in '🂱🃁🃑':
            yield carta

In [15]:
list(Trinca())

['🂱', '🃁', '🃑']

### Outra classe iterável

In [16]:
from random import randint

class Dados:
    def __iter__(self):
        for _ in range(5):
            yield randint(1, 6)

In [17]:
list(Dados())

[3, 4, 3, 4, 4]

### Reinventando `range`

In [18]:
def zrange(n):
    ...

### Reescrevendo `for` com `while`

In [19]:
iterador = iter('ABC')
while True:
    try:
        item = next(iterador)
    except StopIteration:
        del iterador
        break
    print(item)

A
B
C


## A história do yield

Fonte: [Generator Power (11)](https://speakerdeck.com/ramalho/python-generator-power?slide=11)

<img src="liskov-CLU.png" style="width: 640px;"/>

### A linguagem CLU

A definição de um novo tipo é um *cluster* (semelhante a classe, antes de existir OOP).

> Objetos do subtipo devem se comportar da mesma forma que os do supertipo,<br>
> até onde qualquer programa que use objetos do supertipo possa perceber.<br>
> —*Princípio da substituição de Liskov*

Outras características que influenciaram linguagens atuais:

* exceções com `signal/except`
* atribuição paralela: `x,y,z := f(t)`
* iteradores com `yield`

Fonte: [Generator Power (12)](https://speakerdeck.com/ramalho/python-generator-power?slide=12)

<img src="liskov-yield-for.png" style="width: 640px;"/>

## Iteradores/geradores essenciais em Python

### Como gerar uma série de números?

```C
    for(int i = 0; i < stop; i++)
        ...
```

```python
    for i in range(stop):
        ...
```

### Como numerar uma série de itens?

Vamos reinventar `enumerate`:

In [20]:
def zenum(iterável):
    ...

### Como percorrer duas coleções em paralelo?

Vamos reinventar `zip`:

In [21]:
def zzip(a, b):
    ...

## O pacote `itertools` 👀

https://docs.python.org/pt-br/3.13/library/itertools.html

## Bônus: expressões geradoras

In [22]:
s = 'banana'
g = (ord(c) for c in s)
g

<generator object <genexpr> at 0x720a9c15dbe0>

In [23]:
for l in g:
    print(l)

98
97
110
97
110
97


### Um caso de uso

In [24]:
for vogal in (c for c in s if c not in 'aeiou'):
    print(vogal)

b
n
n


### Dados com genexp

In [25]:
class Dados:
    def __iter__(self):
        return (randint(1, 6) for _ in range(5))

In [26]:
d = Dados()

iter(d)

<generator object Dados.__iter__.<locals>.<genexpr> at 0x720a9c0fdfc0>

In [27]:
list(d)

[2, 3, 4, 6, 4]

### Dados simples

In [28]:
list(randint(1, 6) for _ in range(5))

[6, 2, 6, 1, 2]

### Trem com genexp

In [29]:
class Trem2:
    def __init__(self, carros):
        self.carros = carros
    
    def __iter__(self):
        return (f'carro #{n}' for n in range(self.carros))

In [30]:
for carro in trem:
    print(carro)

carro #0
carro #1
carro #2


## Onde me encontrar

| lugar     | endereço |
|----------:|:----------------------|
| Blog      | https://ramalho.org   |
| Mastodon  | @lr@ciberlandia.pt    |
| BlueSky   | @ramalho.org          |
| E-mail    | luciano@ramalho.org   |
| Github    | @ramalho              |

## Muito grato 🙏

In [31]:
print(''.join(chr(n) for n in (79, 75)))

OK
